## Great Barrier Reef - Training Script

Downloads the specified branch of the repo from the github and can run the training or even a full parameter sweep.

In [1]:
#@title Select your options

user = "stuttgartteam8" #@param ["stuttgartteam8", "david", "nick"]
branch_name = "GBR_starfish_detection"    #@param {type:"string"}

sweep = False #@param {type:"boolean"}
sweep_file = "sweep.yaml" #@param {type:"string"}

In [2]:
!nvidia-smi

Wed Dec 29 00:45:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# download dataset
%%capture
%cd /content/
# images 0-478 (part of whole data set)
!gdown https://drive.google.com/uc?id=1nVROGC31eVwThW7xYUM8zuxlmQVgC9xr
!unzip -u "/content/GBR_dataset_partial.zip" -d "/content/";

In [13]:
# delete existing repo to ensure the newest is fetched from github 
% cd /
!rm -r /content/dl-lab-21w-team08

# downlhttps://drive.google.com/uc?id=1nVROGC31eVwThW7xYUM8zuxlmQVgC9xroad repo
# %%capture
%cd /content/
#if user == "david":
git_access_token = "28fdd19659f4a0e5bcaf16dd820e5c68bed91102"

# !git clone https://28fdd19659f4a0e5bcaf16dd820e5c68bed91102@github.tik.uni-stuttgart.de/iss/dl-lab-21w-team08.git
clone_comand = f"git clone --single-branch --branch {branch_name} https://{git_access_token}@github.tik.uni-stuttgart.de/iss/dl-lab-21w-team08.git"
!$clone_comand

/
/content
Cloning into 'dl-lab-21w-team08'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 319 (delta 18), reused 21 (delta 10), pack-reused 287
Receiving objects: 100% (319/319), 5.62 MiB | 28.33 MiB/s, done.
Resolving deltas: 100% (187/187), done.


In [7]:
# install dependencies
%%capture
!pip install wandb
!pip install tensorflow_addons
#!pip install gin-config

In [14]:
%%capture out
# run training script
# 37262d20054e8dbf092705158103cd02e31691d6
%cd /content/
!mkdir wandb
%cd /content/dl-lab-21w-team08/GBR_starfish_detection/
if user == "stuttgartteam8":
    !wandb login 37262d20054e8dbf092705158103cd02e31691d6
elif user == "david":
    !wandb login 37262d20054e8dbf092705158103cd02e31691d6
elif user == "nick":
    !wandb login e9d83c342bbf06922fa978a0d9a92ed847bf4f3e
if sweep:
    configure_sweep = f"wandb sweep {sweep_file}"
    !$configure_sweep

In [ ]:
if sweep == 0:
    !python main.py -p="/content/GBR_dataset_partial/"

    video_id  ...                                        annotations
20         0  ...  [{'x': 555, 'y': 214, 'width': 50, 'height': 32}]
21         0  ...  [{'x': 550, 'y': 214, 'width': 50, 'height': 32}]
22         0  ...  [{'x': 538, 'y': 209, 'width': 56, 'height': 47}]
23         0  ...  [{'x': 535, 'y': 207, 'width': 58, 'height': 48}]
24         0  ...  [{'x': 532, 'y': 205, 'width': 59, 'height': 49}]

[5 rows x 6 columns]
2021-12-29 01:05:10.775571: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Epoch 1/150
6/6 - 8s - loss: 7724897.5000 - val_loss: 1247551.7500 - lr: 0.1000 - 8s/epoch - 1s/step
Epoch 2/150
6/6 - 2s - loss: 63872.6602 - val_loss: 477667.5312 - lr: 0.0955 - 2s/epoch - 406ms/step
Epoch 3/150
6/6 - 2s - loss: 28585.1641 - val_loss: 97132.5000 - lr: 0.0911 - 2s/epoch - 410ms/step
Epoch 4/150
6/6 - 2s - loss: 7059.0288 - val

In [ ]:
from input import load, show_annotated_image
from train import train

from tensorflow import keras
model = keras.models.load_model('model.h5')
ds_train, ds_val, ds_test = load(config)
for x,y in ds_train:
        y_pred = model.predict(x)
        show_annotated_image(config, x[0], y_pred[0], y[0])
        print(f"y_true: {y[0][:,:,0]}")
        print(f"y_pred: {y_pred[0][:,:,0]}")

In [ ]:
if sweep:
    command = out.stdout[out.stdout.find("Run sweep agent with: \x1b[33m") + 27 : -5]
    print(command)
    !$command

wandb agent davidu/diabetic_retinopathy/jlwwq9q0
wandb: Starting wandb agent 🕵️
2021-12-13 14:46:32,386 - wandb.wandb_agent - INFO - Running runs: []
2021-12-13 14:46:32,626 - wandb.wandb_agent - INFO - Agent received command: run
2021-12-13 14:46:32,626 - wandb.wandb_agent - INFO - Agent starting run with config:
	architecture: resnet50
	augment_crop: 400
	balancing: True
	batch_size: 16
	data_dir: /content/IDRID_dataset/
	dense0: 256
	dense1: 64
	dense2: 64
	dense_prep: gap
	dropout: 0.1
	epochs: 50
	fine_tuning: False
	fine_tuning_epochs: 5
	fine_tuning_learning_rate: 0.001
	img_width: 512
	input_shape: [224, 224, 3]
	learning_rate: 0.001
	learning_rate_decay: 10
	n_classes: 5
	optimizer: adam
	reg_lambda: 0.01
	val_split: 0.8
	w_init_HeNormal: True
2021-12-13 14:46:32,630 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python main.py --log_wandb=online
wandb: Currently logged in as: davidu (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.i